# Deep learning Assignment - Demo notebook

System specifications:
- Created on google colab

Overview:
This notebook is the demo notebook where we just load select models and run evaluation on them. The creation of the models is described in the seperate notebooks.

##### FAQ:
- If getting an access error when downloading models, please enable gpu in colab runtime before trying again. If issue still persists, please contact.

### Imports

In [2]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords
import string

from sklearn.model_selection import train_test_split

import tensorflow as tf
import numpy as np

In [3]:
import tensorflow_hub as hub

import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Embedding, Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D, LSTM, SimpleRNN, MaxPooling1D, Conv1D, TimeDistributed, AveragePooling1D
from tensorflow.keras.datasets import imdb
import tensorflow as tf

import plotly.express as px
import matplotlib.pyplot as plt

In [4]:
from sklearn.preprocessing import OneHotEncoder
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score

In [5]:
from keras.preprocessing.text import Tokenizer

In [6]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [7]:
# credentials to get the data file
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [8]:
import gdown

### Get the files with the test records from Google drive

In [9]:
# https://drive.google.com/file/d/1Rrnsq2henQapAItUVlIBxUeqfGw8CNi3/view?usp=sharing
guardTestData = drive.CreateFile({'id':'1Rrnsq2henQapAItUVlIBxUeqfGw8CNi3'})
guardTestData.GetContentFile('guardian_testrecords.csv')

In [10]:
guard_test_df = pd.read_csv("guardian_testrecords.csv", delimiter =",", index_col=False, )
display(guard_test_df)

print(guard_test_df["bodyContent"][0])

guard_test_df['bodyContent'] = guard_test_df['bodyContent'].apply(lambda a: str(a).encode('ascii','ignore'))
guard_test_df['bodyContent'] = guard_test_df['bodyContent'].apply(lambda a: a.decode('ascii','ignore'))

,Unnamed: 0,bodyContent,sectionName
0,144813,The Guardian is looking into thefts of cryptoc...,Technology
1,94897,Most days you cant walk through San Franciscos...,World news
2,1132,One of the many fascinating aspects of John Se...,Music
3,106414,Congressional Democrats have launched contempt...,US news
4,15691,.,Sport
...,...,...,...
1940,19991,"It was, said Nick Clegg, something youd expect...",UK news
1941,38321,Maurice the French cockerel fancies himself as...,World news
1942,101198,Expectations continue to grow over the prospec...,Football
1943,101267,"Black, Asian and minority ethnic people in Eng...",World news


The Guardian is looking into thefts of cryptocurrency and digital assets


In [11]:
guard_X_test = guard_test_df["bodyContent"]

# It's fine to encode without fitting to training because we have verified that the top 20 are there in both training and test
oh = OneHotEncoder()
guard_Y_test = pd.get_dummies(guard_test_df["sectionName"])
display(guard_Y_test)

print(len(guard_Y_test))

,Art and design,Australia news,Books,Business,Environment,Film,Football,Life and style,Media,Music,Opinion,Politics,Society,Sport,Stage,Technology,Television & radio,UK news,US news,World news
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1940,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1942,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1943,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


1945


In [12]:
# https://drive.google.com/file/d/10beQppb8BD4gV09d01MOcjiB8baGHcBp/view?usp=sharing
bbcTestData = drive.CreateFile({'id':'10beQppb8BD4gV09d01MOcjiB8baGHcBp'})
bbcTestData.GetContentFile('bbc_testrecords.csv')

In [13]:
bbc_test_df = pd.read_csv("bbc_testrecords.csv", delimiter =",", index_col=False, )
display(bbc_test_df)
#Note: While saving the test data for demo, made the column names same as guardian (Ref: Part 2 notebook)

,Unnamed: 0,bodyContent,sectionName
0,920,oscar nominees lack pulling power this year s ...,entertainment
1,2149,wi-fi web reaches farmers in peru a network of...,tech
2,1710,stevens named in england line-up england have ...,sport
3,97,ireland win eclipses refereeing errors the i...,sport
4,1343,melzer shocks agassi in san jose second seed a...,sport
...,...,...,...
440,108,hong kong in 2011 world cup bid hong kong is h...,sport
441,304,india s rupee hits five-year high india s rupe...,business
442,2210,teens know little of politics teenagers ques...,politics
443,2154,green reports shun supply chain nearly 20% mor...,business


In [14]:
bbc_X_test = bbc_test_df["bodyContent"]
# It's fine to encode without fitting to training because we have verified that the top 20 are there in both training and test
oh = OneHotEncoder()
bbc_Y_test = pd.get_dummies(bbc_test_df["sectionName"])
display(bbc_Y_test)

,business,entertainment,politics,sport,tech
0,0,1,0,0,0
1,0,0,0,0,1
2,0,0,0,1,0
3,0,0,0,1,0
4,0,0,0,1,0
...,...,...,...,...,...
440,0,0,0,1,0
441,1,0,0,0,0
442,0,0,1,0,0
443,1,0,0,0,0


### Get the models for the first part from google drive

### Getting model 4 of part 1 from Google drive

In [15]:
#Getting model 4
#https://drive.google.com/drive/folders/1Q5wdXwa4PVQhG1dJc8gfr-i2Nyv_adse?usp=share_link
part1_model4_url = "https://drive.google.com/drive/folders/1Q5wdXwa4PVQhG1dJc8gfr-i2Nyv_adse?usp=share_link"
gdown.download_folder(part1_model4_url, quiet=True, use_cookies=False)

['/content/model4save/assets/tokens.txt',
 '/content/model4save/fingerprint.pb',
 '/content/model4save/keras_metadata.pb',
 '/content/model4save/saved_model.pb',
 '/content/model4save/variables/variables.data-00000-of-00001',
 '/content/model4save/variables/variables.index']

In [16]:
part1_model4 = tf.keras.models.load_model('model4save')
part1_model4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense_13 (Dense)            (None, 400)               8400      
                                                                 
 dense_14 (Dense)            (None, 400)               160400    
                                                                 
 dense_15 (Dense)            (None, 400)               160400    
                                                                 
 dense_16 (Dense)            (None, 400)               160400    
                                                                 
 dense_17 (Dense)            (None, 20)                8020      
                                                                 
Total params: 897,640
Trainable params: 497,620
Non-tr

In [17]:
p1_m4_acc = part1_model4.evaluate(guard_X_test, guard_Y_test, batch_size=32)

61/61 [==============================] - 3s 5ms/step - loss: 3.8287 - accuracy: 0.3815


#### Getting model 6 of part 1 from Google drive

In [18]:
#Getting model 6
# https://drive.google.com/drive/folders/1jqnW2a0nsA2Bju7x9Nc5g2b_g6BMcRpX?usp=sharing
part1_model6_url = "https://drive.google.com/drive/folders/1jqnW2a0nsA2Bju7x9Nc5g2b_g6BMcRpX?usp=sharing"
gdown.download_folder(part1_model6_url, quiet=True, use_cookies=False)

['/content/model6save/fingerprint.pb',
 '/content/model6save/keras_metadata.pb',
 '/content/model6save/saved_model.pb',
 '/content/model6save/variables/variables.data-00000-of-00001',
 '/content/model6save/variables/variables.index']

In [19]:
part1_model6 = tf.keras.models.load_model('model6save')
part1_model6.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding_4 (Embedding)     (None, None, 16)          1120000   
                                                                 
 conv1d_1 (Conv1D)           (None, None, 80)          3920      
                                                                 
 global_average_pooling1d_1   (None, 80)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_21 (Dense)            (None, 400)               32400     
                                                                 
 dense_22 (Dense)            (None, 400)              

In [20]:
p1_m6_acc = part1_model6.evaluate(guard_X_test, guard_Y_test, batch_size=32)

61/61 [==============================] - 6s 21ms/step - loss: 9.8564 - accuracy: 0.3429


### Getting model 7 of Part 1 from Google drive

In [21]:
#Getting model 7
# https://drive.google.com/drive/folders/1jIf5UHSYrXjmEQmxrNljIf75l1bQFI80?usp=share_link
part1_model7_url = "https://drive.google.com/drive/folders/1jIf5UHSYrXjmEQmxrNljIf75l1bQFI80?usp=share_link"
gdown.download_folder(part1_model7_url, quiet=True, use_cookies=False)

['/content/model7save/fingerprint.pb',
 '/content/model7save/keras_metadata.pb',
 '/content/model7save/saved_model.pb',
 '/content/model7save/variables/variables.data-00000-of-00001',
 '/content/model7save/variables/variables.index']

In [22]:
part1_model7 = tf.keras.models.load_model('model7save')
part1_model7.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding_6 (Embedding)     (None, None, 16)          1120000   
                                                                 
 conv1d_5 (Conv1D)           (None, None, 80)          25680     
                                                                 
 average_pooling1d_2 (Averag  (None, None, 80)         0         
 ePooling1D)                                                     
                                                                 
 conv1d_6 (Conv1D)           (None, None, 80)          64080     
                                                                 
 average_pooling1d_3 (Averag  (None, None, 80)        

In [23]:
p1_m7_acc = part1_model7.evaluate(guard_X_test, guard_Y_test, batch_size=32)

61/61 [==============================] - 5s 51ms/step - loss: 4.3046 - accuracy: 0.3933


### Getting transfer model 1 of Part 2 from google drive

In [24]:
#Getting transfer model 1
# https://drive.google.com/drive/folders/1lueThJhBft696XxjyAvFGIqNMVPDPY5a?usp=share_link
part2_tfmodel1_url = "https://drive.google.com/drive/folders/1lueThJhBft696XxjyAvFGIqNMVPDPY5a?usp=share_link"
gdown.download_folder(part2_tfmodel1_url, quiet=True, use_cookies=False)

['/content/tfmodel1save/fingerprint.pb',
 '/content/tfmodel1save/keras_metadata.pb',
 '/content/tfmodel1save/saved_model.pb',
 '/content/tfmodel1save/variables/variables.data-00000-of-00001',
 '/content/tfmodel1save/variables/variables.index']

In [25]:
part2_model1 = tf.keras.models.load_model('tfmodel1save')
part2_model1.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding_4 (Embedding)     (None, None, 16)          1120000   
                                                                 
 conv1d_1 (Conv1D)           (None, None, 80)          3920      
                                                                 
 global_average_pooling1d_1   (None, 80)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_21 (Dense)            (None, 400)               32400     
                                                                 
 dense_22 (Dense)            (None, 400)              

In [26]:
p2_m1_acc = part2_model1.evaluate(bbc_X_test, bbc_Y_test, batch_size=32)

14/14 [==============================] - 1s 32ms/step - loss: 0.5833 - accuracy: 0.8809


### Getting transfer model 5 of part 2 from google drive

In [27]:
#Getting transfer model 5
#https://drive.google.com/drive/folders/1hsBaCD4OjVi4250y7g7VZvkVfyLhVNFi?usp=share_link
part2_tfmodel1_url = "https://drive.google.com/drive/folders/1hsBaCD4OjVi4250y7g7VZvkVfyLhVNFi?usp=share_link"
gdown.download_folder(part2_tfmodel1_url, quiet=True, use_cookies=False)

['/content/tfmodel5save/fingerprint.pb',
 '/content/tfmodel5save/keras_metadata.pb',
 '/content/tfmodel5save/saved_model.pb',
 '/content/tfmodel5save/variables/variables.data-00000-of-00001',
 '/content/tfmodel5save/variables/variables.index']

In [28]:
part2_model5 = tf.keras.models.load_model('tfmodel5save')
part2_model5.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, None, 16)          1120000   
                                                                 
 conv1d (Conv1D)             (None, None, 80)          25680     
                                                                 
 average_pooling1d (AverageP  (None, None, 80)         0         
 ooling1D)                                                       
                                                                 
 conv1d_1 (Conv1D)           (None, None, 80)          64080     
                                                                 
 average_pooling1d_1 (Averag  (None, None, 80)        

In [29]:
p2_m5_acc = part2_model5.evaluate(bbc_X_test, bbc_Y_test, batch_size=32)

14/14 [==============================] - 2s 72ms/step - loss: 1.6094 - accuracy: 0.2157


### Getting the custom generator model from part 3

In [30]:
#Getting the part 3 generator model
#https://drive.google.com/drive/folders/1-I4cawHms40KFfKze3vnliP6T475t8Xb?usp=share_link
part3_genmodel_url = "https://drive.google.com/drive/folders/1-I4cawHms40KFfKze3vnliP6T475t8Xb?usp=share_link"
gdown.download_folder(part3_genmodel_url, quiet=True, use_cookies=False)

['/content/gen_model/fingerprint.pb',
 '/content/gen_model/keras_metadata.pb',
 '/content/gen_model/saved_model.pb',
 '/content/gen_model/variables/variables.data-00000-of-00001',
 '/content/gen_model/variables/variables.index']

In [31]:
part3_model = tf.keras.models.load_model('gen_model')

In [32]:
# https://drive.google.com/file/d/1WGrz9e8mGh3Ktols0gc9p0y-UcjCLD4-/view?usp=share_link
encodingsFile = drive.CreateFile({'id':'1WGrz9e8mGh3Ktols0gc9p0y-UcjCLD4-'})
encodingsFile.GetContentFile('encodings.csv')

enc_data = pd.read_csv("encodings.csv", delimiter =",", index_col=False, )
display(enc_data)

,Unnamed: 0,columns
0,0,0
1,1,00
2,2,000
3,3,000m
4,4,000seat
...,...,...
11252,11252,zimbabwe
11253,11253,zinc
11254,11254,zogbia
11255,11255,zone


In [33]:
seed_word = ["Face"]
final_word_list = ["Face"]
column_headers = list(enc_data["columns"].values)

for i in range(0,20):
    pred = part3_model.predict(seed_word)
    word_index = np.argmax(pred, axis=1)
    #word = str(word_Y_le.columns[word_index].value)
    word = column_headers[int(word_index)]
    print(word)
    final_word_list.append(word)
    temp_list = []
    temp_list.append(word)
    seed_word = temp_list

#print(final_word_list)

print("The generated sentence is:")
print(" ".join(final_word_list))

1/1 [==============================] - 1s 522ms/step
the
1/1 [==============================] - 0s 42ms/step
year
1/1 [==============================] - 0s 42ms/step
in
1/1 [==============================] - 0s 45ms/step
the
1/1 [==============================] - 0s 65ms/step
year
1/1 [==============================] - 0s 45ms/step
in
1/1 [==============================] - 0s 47ms/step
the
1/1 [==============================] - 0s 51ms/step
year
1/1 [==============================] - 0s 40ms/step
in
1/1 [==============================] - 0s 44ms/step
the
1/1 [==============================] - 0s 53ms/step
year
1/1 [==============================] - 0s 40ms/step
in
1/1 [==============================] - 0s 46ms/step
the
1/1 [==============================] - 0s 43ms/step
year
1/1 [==============================] - 0s 40ms/step
in
1/1 [==============================] - 0s 43ms/step
the
1/1 [==============================] - 0s 42ms/step
year
1/1 [==============================] - 0s 45ms